In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.linalg import lstsq
from matplotlib.lines import Line2D
import cv2
from scipy.linalg import null_space

In [ ]:
import numpy as np
import cv2
from skimage import io
import matplotlib.pyplot as plt

# Function to calculate the homography matrix from measured points to original points
def calculate_homography(measured_pts, original_pts):
    # Create matrices A and b in homogeneous coordinates
    A = np.zeros((8, 8))
    b = np.zeros(8)
    for i in range(4):
        x, y = original_pts[i]
        x_meas, y_meas = measured_pts[i]
        A[2 * i] = [x, y, 1, 0, 0, 0, -x * x_meas, -y * x_meas]
        A[2 * i + 1] = [0, 0, 0, x, y, 1, -x * y_meas, -y * y_meas]
        b[2 * i], b[2 * i + 1] = x_meas, y_meas

    # Solve for the homography vector and reshape into matrix form
    homography_vector = np.linalg.solve(A, b)
    homography_matrix = np.vstack([homography_vector, 1]).reshape(3, 3)
    return homography_matrix

# Function to calculate new image dimensions and offsets
def calculate_image_dimensions(corner_pts):
    min_x, min_y = corner_pts.min(axis=0)
    max_x, max_y = corner_pts.max(axis=0)
    height_new, width_new = max_y - min_y + 1, max_x - min_x + 1
    offset_y, offset_x = -min_y, -min_x
    return int(height_new), int(width_new), int(offset_y), int(offset_x)

# Function to apply inverse homography to an image
def apply_inverse_homography(H, img, new_img_dims):
    orig_height, orig_width = img.shape[:2]
    new_height, new_width, offset_y, offset_x = new_img_dims

    # Create coordinate grid for new image
    y_indices, x_indices = np.indices((new_height, new_width))
    homogeneous_coords = np.stack([(x_indices - offset_x).ravel(), (y_indices - offset_y).ravel(), np.ones_like(x_indices).ravel()])

    # Apply the inverse homography
    H_inv = np.linalg.inv(H)
    transformed_coords = H_inv @ homogeneous_coords
    transformed_coords /= transformed_coords[2, :]

    x_transformed = np.round(transformed_coords[0]).astype(int)
    y_transformed = np.round(transformed_coords[1]).astype(int)

    # Construct the transformed image
    transformed_image = np.zeros((new_height, new_width, 3), dtype=img.dtype)
    valid_mask = (0 <= x_transformed) & (x_transformed < orig_width) & (0 <= y_transformed) & (y_transformed < orig_height)
    transformed_image[y_indices.ravel()[valid_mask], x_indices.ravel()[valid_mask]] = img[y_transformed[valid_mask], x_transformed[valid_mask]]

    return transformed_image

# Function to calculate the line passing through two points
def calculate_line_through_points(point1, point2):
    p1_hom = np.array([point1[0], point1[1], 1])
    p2_hom = np.array([point2[0], point2[1], 1])
    line = np.cross(p1_hom, p2_hom)
    line /= np.linalg.norm(line)
    return line

# Correct projective distortion based on provided points
def correct_projective_distortion(points, image, name):
    lines = [calculate_line_through_points(points[i], points[(i + 1) % 4]) for i in range(4)]
    vanishing_points = [np.cross(lines[i], lines[(i + 2) % 4]) for i in range(2)]
    vanishing_line = np.cross(vanishing_points[0], vanishing_points[1])
    vanishing_line /= np.linalg.norm(vanishing_line)

    homography = np.eye(3)
    homography[2] = vanishing_line
    homography /= homography[2, 2]

    result_img = apply_inverse_homography(homography, image, calculate_image_dimensions(find_corners(homography, image)))
    plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    print(f'Homography for {name}:\n', homography)
    return lines, homography

# Function to transform lines using a given homography
def transform_lines(lines, homography):
    inv_homography_t = np.linalg.inv(homography).T
    return [np.dot(inv_homography_t, line) / np.linalg.norm(np.dot(inv_homography_t, line)) for line in lines]

# Compute the S matrix for affine correction
def compute_S_matrix(transformed_lines):
    A = np.array([[transformed_lines[0][0] * transformed_lines[1][0], 
                   transformed_lines[0][0] * transformed_lines[1][1] + transformed_lines[0][1] * transformed_lines[1][0]],
                  [transformed_lines[4][0] * transformed_lines[5][0], 
                   transformed_lines[4][0] * transformed_lines[5][1] + transformed_lines[4][1] * transformed_lines[5][0]]])
    b = -np.array([transformed_lines[0][1] * transformed_lines[1][1], transformed_lines[4][1] * transformed_lines[5][1]])
    s_values = np.linalg.solve(A, b)
    S_matrix = np.array([[s_values[0], s_values[1]], [s_values[1], 1]])
    return S_matrix

# Derive homography from matrix S
def derive_homography_from_S(S_matrix):
    u, s, vh = np.linalg.svd(S_matrix)
    sqrt_eigenvalues = np.sqrt(np.diag(s))
    transformation_matrix = vh @ sqrt_eigenvalues @ vh.T
    affine_homography = np.eye(3)
    affine_homography[:2, :2] = transformation_matrix
    return affine_homography

# Correct affine distortion
def correct_affine_distortion(projective_lines, projective_homography, image, name):
    affine_lines = transform_lines(projective_lines, projective_homography)
    S_matrix = compute_S_matrix(affine_lines)
    undist_homography = derive_homography_from_S(S_matrix)
    combined_homography = np.linalg.inv(undist_homography) @ projective_homography
    combined_homography /= combined_homography[2, 2]

    result_img = apply_inverse_homography(combined_homography, image, calculate_image_dimensions(find_corners(combined_homography, image)))
    plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    print(f'Homography for affine correction ({name}):\n', undist_homography)
    print(f'Homography for {name}:\n', combined_homography)

# Compute dual conic from lines
def compute_dual_conic(lines):
    A = np.zeros((5, 5), dtype=float)
    b = np.zeros(5, dtype=float)
    for i in range(3):
        A[i] = [lines[i][0] * lines[i + 1][0],
                lines[i][1] * lines[i + 1][0] + lines[i][0] * lines[i + 1][1],
                lines[i][1] * lines[i + 1][1],
                lines[i][2] * lines[i + 1][0] + lines[i][0] * lines[i + 1][2],
                lines[i][2] * lines[i + 1][1] + lines[i][1] * lines[i + 1][2]]
        b[i] = -lines[i][2] * lines[i + 1][2]
    A[3] = [lines[3][0] * lines[0][0],
            lines[3][1] * lines[0][0] + lines[3][0] * lines[0][1],
            lines[3][1] * lines[0][1],
            lines[3][2] * lines[0][0] + lines[3][0] * lines[0][2],
            lines[3][2] * lines[0][1] + lines[3][1] * lines[0][2]]
    b[3] = -lines[3][2] * lines[0][2]
    A[4] = [lines[4][0] * lines[5][0],
            lines[4][1] * lines[5][0] + lines[4][0] * lines[5][1],
            lines[4][1] * lines[5][1],
            lines[4][2] * lines[5][0] + lines[4][0] * lines[5][2],
            lines[4][2] * lines[5][1] + lines[4][1] * lines[5][2]]
    b[4] = -lines[4][2] * lines[5][2]
    dual_conic = np.linalg.solve(A, b)
    return dual_conic

# Perform one-step correction
def perform_one_step_correction(points, image, name, scale_factor=1):
    lines = [calculate_line_through_points(points[i], points[(i + 1) % 4]) for i in range(4)] + \
            [calculate_line_through_points(points[1], points[3]), calculate_line_through_points(points[0], points[2])]
    dual_conic = compute_dual_conic(lines)
    u, s, vh = np.linalg.svd([[dual_conic[0], dual_conic[1]], [dual_conic[1], dual_conic[2]]])
    sqrt_eigenvalues = np.sqrt(np.diag(s))
    transformation_matrix = vh @ sqrt_eigenvalues @ vh.T
    affine_vector = np.linalg.inv(transformation_matrix) @ np.array([dual_conic[3], dual_conic[4]])
    combined_homography = np.eye(3)
    combined_homography[:2, :2] = transformation_matrix
    combined_homography[2, :2] = affine_vector
    visualize_transformation(image, np.linalg.inv(combined_homography), name, scale_factor)
    np.set_printoptions(suppress=True)
    print(f'Homography for {name}:\n', np.linalg.inv(combined_homography) / np.linalg.inv(combined_homography)[2, 2])
    np.set_printoptions(suppress=False)

# Read images
image1 = cv2.imread('./HW3_images/corridor.jpeg')
image2 = cv2.imread('./HW3_images/board_1.jpeg')

# Source points
corridor_points = np.array([[1084, 532], [1305, 491], [1076, 1215], [1296, 1332]])
board_points = np.array([[72, 427], [425, 1798], [1220, 146], [1349, 1942]])

# Correct projective distortion from image 1
proj_lines_img1, proj_H_img1 = correct_projective_distortion(corridor_points, image1, 'corridor_projective')

# Correct affine distortion from image 1 using the previously obtained homography matrix
correct_affine_distortion(proj_lines_img1, proj_H_img1, image1, 'corridor_affine')
